In [ ]:
import backtrader as bt
import numpy as np
import datetime as dt
from datetime import datetime

class Comminfo(bt.Strategy):
    params = (
            ('Printlog', True),
        )
    def log(self, txt, dt=None):
        print(txt)
    def __init__(self):
        #self.morningstart=datetime.now().replace(hours=9, minut=25, second=0, microsecond=0) # morning starting time
        #self.eveningend=datetime.now().replace(hours=14, minute=40, second=0, microsecond=0) # Evening ending time
        #self.timestop=datetime.now().replace(hours=15, minute=0, second=0, microsecond=0)    # Stop taking the Orders
        self.morning_start=dt.time(hours=9, mintue=25, second=0, microsecond=0)   # Here we can follow above 3 steps or this 3 steps
        self.evenong_end=dt.time(hours=14, minute=40, second=0, microsecond=0)
        self.time_stop=dt.time(hours=14, minute=40, second=0, microsecond=0)
        
        self.ORH = None
        self.ORL = None
        
    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            return
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))
                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))
            self.bar_executed = len(self)
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')
        self.order = None
        
    def notify_trade(self, trade):
        if not trade.isclosed:
            return
        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))
        
    def next(self):
        print("Current time: ", self.data.datetime.time(0))
        print("Current date: ", self.data.datetime.date(0))
        
        if self.data.datetime(0).hours == 9 and self.data.datetime.time(0).minute==25:
            self.ORH = max(self.data.high[0], self.data.high[-1], self.data.high[-2])   # checking the High breaking
            self.ORL = min(self.data.Low[0], self.data.Low[-1], self.data.low[-2])      # checking the Low breaking
            print("ORH: ", self.ORH, "ORL: ",self.ORL)
        
        if not self.position:
            if self.data.datetime.time(0) > self.morning_start and self.data.datetime(0) < self.evening_end:
                if self.data.close[0] > self.ORH:
                    self.order = self.buy()
                    self.stoporder = self.sell(exectype=bt.Order.Stop, Price=self.ORL)
                if self.data.close[0] < self.ORL:
                    self.order = self.sell()
                    self.stoporder = self.buy(exectype=bt.Order.Stop, Price=self.ORH)
        else:
            if self.data.datetime.time(0)==self.time_stop:
                self.cancel(self.stoporder)
                self.close()
    def stop(self):
        self.log(' Ending value %.2f' % (self.broker.getvalue()))
        
if __name__=='__main__':
    cerebro = bt.Cerebro(stdstats = True)
    comminfo = CommInfo(
            commission = 0.001,
            mult = 75,
            margin = 100000)
    
    cerebro.broker.addcommissioninfo(comminfo)
    
    datapath = "../Data/CLEANED DATA/Nifty_F1_5m_data_clean.csv"
    data = bt.feeds.GenericCSVData(
        dataname=datapath,
        fromdate=datetime.datetime(2019,1,1),
        todate=datetime.datetime(2021,7,21),
        datetime= 0,
        timeframe = bt.TimeFrame.Days,
        compression = 1,
        dtformat = ('%Y-%m-%d-%H-%M'),                 
        Open = 1,
        high = 2,
        low = 3,
        close = 4,
        volume = None,
        openinterest = None,
        reverse=False,
        header=0)

    cerebro.adddata(data)
    cerebro.addsizer(bt.sizers.FixedSize, stake=1)                  
    cerebro.broker.setcommission(commission=0.001)                     
    cerebro.broker.setcash(100000.0)
    print('Starting Portfolio Value: ', cerebro.broker.getvalue())
    cerebro.run()
    print('Final Portfolio Value: ', cerebro.broker.getvalue())
               

In [ ]:
import config
import matplotlib
import backtrader, pandas, sqlite3
from datetime import date, datetime, time, timedelta
import backtrader as bt

class OpeningRangeBreakout(backtrader.Strategy):
    params = dict(
        num_opening_bars=15
    )

    def __init__(self):
        self.opening_range_low = 0
        self.opening_range_high = 0
        self.opening_range = 0
        self.bought_today = False
        self.order = None
    
    def log(self, txt, dt=None):
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))
        
    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            return
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))
            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')
        self.order = None

    def next(self):
        current_bar_datetime = self.data.num2date(self.data.datetime[0])
        previous_bar_datetime = self.data.num2date(self.data.datetime[-1])

        if current_bar_datetime.date() != previous_bar_datetime.date():
            self.opening_range_low = self.data.low[0]
            self.opening_range_high = self.data.high[0]
            self.bought_today = False
        
        opening_range_start_time = time(9, 30, 0)
        dt = datetime.combine(date.today(), opening_range_start_time) + timedelta(minutes=self.p.num_opening_bars)
        opening_range_end_time = dt.time()

        if current_bar_datetime.time() >= opening_range_start_time \
            and current_bar_datetime.time() < opening_range_end_time:           
            self.opening_range_high = max(self.data.high[0], self.opening_range_high)
            self.opening_range_low = min(self.data.low[0], self.opening_range_low)
            self.opening_range = self.opening_range_high - self.opening_range_low
        else:
            if self.order:
                return
            
            if self.position and (self.data.close[0] > (self.opening_range_high + self.opening_range)):
                self.close()
                
            if self.data.close[0] > self.opening_range_high and not self.position and not self.bought_today:
                self.order = self.buy()
                self.bought_today = True

            if self.position and (self.data.close[0] < (self.opening_range_high - self.opening_range)):
                self.order = self.close()

            if self.position and current_bar_datetime.time() >= time(15, 45, 0):
                self.log("RUNNING OUT OF TIME - LIQUIDATING POSITION")
                self.close()

    def stop(self):
        self.log('(Num Opening Bars %2d) Ending Value %.2f' %
                 (self.params.num_opening_bars, self.broker.getvalue()))

        if self.broker.getvalue() > 130000:
            self.log("*** BIG WINNER ***")

        if self.broker.getvalue() < 70000:
            self.log("*** MAJOR LOSER ***") 

if __name__ == '__main__':
    cerebro = bt.Cerebro()
    cerebro.addstrategy(OpeningRangeBreakout)
    datapath = "../../Data/TCS.NS.csv"
    data = bt.feeds.YahooFinanceCSVData(
        dataname=datapath,
        fromdate=datetime(2003, 2, 1),
        todate=datetime(2010, 12, 31),
        reverse=False)
    cerebro.adddata(data)
    cerebro.addstrategy(OpeningRangeBreakout)
    #strats = cerebro.optstrategy(OpeningRangeBreakout, num_opening_bars=[15, 30, 60])
    cerebro.broker.setcash(100000.0)
    cerebro.addsizer(bt.sizers.FixedSize, stake=10)
    cerebro.broker.setcommission(commission=0.0)
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
    cerebro.run()
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
    #cerebro.plot()
   
